<a href="https://colab.research.google.com/github/kangmg/molecule_aligner/blob/main/example_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kangmg/molecule_aligner.git

Cloning into 'molecule_aligner'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 94 (delta 23), reused 86 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (94/94), 1.63 MiB | 11.78 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [17]:
%cd molecule_aligner
!uv pip install . -q
%cd images

/content/molecule_aligner
/content/molecule_aligner/images


In [18]:
from ase.io import Trajectory
from ase.io import write

traj_34b = Trajectory('3_4_b.traj')
traj_76d = Trajectory('7_6_d.traj')

write('34a.traj', traj_34b[::5])
write('76d.traj', traj_76d[::6])

In [19]:
from molecule_aligner import build_reaction_pathway
from ase.io import read

# metal complex (bpy, 3*CO)
base_idx = [
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
    15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
    28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
    41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
    54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
    67, 68, 69, 70
    ]

protonated_idx = [
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
    15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
    28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
    41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
    54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
    67, 68, 69, 70, 71
    ]

steps = [
        # 1) initial frames
        {
            'type': 'frame',
            'source': '2.xyz',
            'repeat': 3,
        },
        # 2) smoothing to elongated (NR3H+)-ML5
        {
            'type': 'interpolate',
            'from': '2.xyz',
            'to': read('a.xyz', format='xyz')[base_idx],
            'frames': 5,
            'method': 'linear'
        },
        # 3) HNR3+ complexiation
        {
            'type': 'interpolate',
            'from': 'a.xyz',
            'to': '3.xyz',
            'frames': 20,
            'method': 'linear'
        },
        # 4) smoothing btw opt - irc images
        {
            'type': 'interpolate',
            'from': '3.xyz',
            'to': read('34a.traj', index=0),
            'frames': 5,
            'method': 'linear'
        },
        # 5) proton transfer
        {
            'type': 'trajectory',
            'source': '34a.traj',
            'frames': 'all',
            'reverse': False
        },
        # 6) smoothing
        {
            'type': 'interpolate',
            'from': read('34a.traj', index=-1),
            'to': 'b.xyz',
            'frames': 5,
            'method': 'linear'
        },
        # 7) amine decomplexiation
        {
            'type': 'interpolate',
            'from': 'b.xyz',
            'to': 'c.xyz',
            'frames': 20,
            'method': 'linear'
        },
        # 8) smoothing elongated to protonated ML5
        {
            'type': 'interpolate',
            'from': read('c.xyz')[protonated_idx],
            'to': '5.xyz',
            'frames': 5,
            'method': 'linear'
        },
        # 9) delay
        {
            'type': 'frame',
            'source': '5.xyz',
            'repeat': 2,
        },
        # 10) smoothing
        {
            'type': 'interpolate',
            'from': '5.xyz',
            'to': read('pre_d.xyz')[protonated_idx],
            'frames': 5,
            'method': 'linear'
        },
        # 11) co2 approaching
        {
            'type': 'interpolate',
            'from': 'pre_d.xyz',
            'to': 'd.xyz',
            'frames': 20,
            'method': 'linear'
        },
        # 12) co2 to acetate rxn
        {
            'type': 'trajectory',
            'source': '76d.traj',
            'reverse': True,
            'frames': 'all',
        },
        # 13) acetate decomplexation
        {
            'type': 'interpolate',
            'from': '7.xyz',
            'to': 'e.xyz',
            'frames': 20,
            'method': 'linear'
        }
    ]

user_workflow = build_reaction_pathway(
    steps=steps,
    base_indices=base_idx,
    output_path='full_path.extxyz'
)

🚀 Building Reaction Pathway with 13 steps
🎯 Alignment indices: 71 atoms

📍 Step 1/13: frame
   📄 Added frame, repeat 3 times

📍 Step 2/13: interpolate
   🔄 Interpolating 5 frames using linear
       From: 71 atoms (C27H36MnN2O5)
       To:   71 atoms (C27H36MnN2O5)

📍 Step 3/13: interpolate
   🔄 Interpolating 20 frames using linear
       From: 97 atoms (C33H52MnN3O8)
       To:   97 atoms (C33H52MnN3O8)

📍 Step 4/13: interpolate
   🔄 Interpolating 5 frames using linear
       From: 97 atoms (C33H52MnN3O8)
       To:   97 atoms (C33H52MnN3O8)

📍 Step 5/13: trajectory
   📂 Loaded trajectory: 22 frames

📍 Step 6/13: interpolate
   🔄 Interpolating 5 frames using linear
       From: 97 atoms (C33H52MnN3O8)
       To:   97 atoms (C33H52MnN3O8)

📍 Step 7/13: interpolate
   🔄 Interpolating 20 frames using linear
       From: 97 atoms (C33H52MnN3O8)
       To:   97 atoms (C33H52MnN3O8)

📍 Step 8/13: interpolate
   🔄 Interpolating 5 frames using linear
       From: 72 atoms (C27H37MnN2O5)
     

In [23]:
import sys
import os

if 'google.colab' in sys.modules:
    os.system('uv pip install git+https://github.com/kangmg/aseview.git')


In [24]:
from aseview import viewer

viewer(user_workflow, atom_scale=0.7, bg_color='#ffffff')